In [123]:
import hashlib

class Node:
    """
    Nodes imitate physical nodes in a hash ring.
    There are NO virtual nodes in this example.
    """
    def __init__(self, identifier: int) -> None:
        self.identifier = identifier
        self._data: str[int, str] = {}
    
    @property
    def elements(self) -> int:
        return len(self._data)

    def set(self, key: int, value: str) -> None:
        self._data[key] = value

    def get(self, key: int) -> str:
        return self._data.get(key, "NaN")

    def remove(self, key: int) -> str:
        self._data.pop(key)
    
    def bulk_pairs(self, upper_bound: int) -> list[tuple[int, str]]:
        pairs: list[tuple[int, str]] = []
        for key, value in self._data.items():
            if key <= upper_bound:
                pairs.append((key, value))
        return pairs


class HashRing:
    """
    Roughly speaking, this is BST to store Nodes and manage the hash.
    aka ClusterOfNodes
    """

    def __init__(self, capacity: int) -> None:
        self._capacity = capacity
        self._nodes: list[Node] = []  # instead of BST let's use list for simplicity

    def _hash_function(self, value: str) -> int:
        """
        In a perfect case - ideally uniform hash function.
        
        In this implementation we trim it by the user-defined capacity (should be infinite in a perfect case) so we have
        lower (0) and upper (capacity) bounds for a hash function and can plan nodes accordingly.
        """
        return int(hashlib.md5(value.encode("utf8")).hexdigest(), 16) % self._capacity

    def add_node(self, identifier: int) -> None:
        """
        Add a new node and rebalance the tree.
        """
        # in a canonical implementation, we have to hash the node identifier (IP, unique id etc.) with the hash function
        # and place it on a ring; here we'll be using just identifier that is bound [0, self._capacity]
        assert 0 <= identifier <= self._capacity

        node = Node(identifier)
        # rebalance the data between nodes
        for existing_node in self._nodes:
            assert node.identifier != existing_node.identifier  # avoiding duplicates
            if node.identifier > existing_node.identifier:
                continue
            for key, value in existing_node.bulk_pairs(node.identifier):
                node.set(key, value)
                existing_node.remove(key)
            break

        # adding node
        self._nodes.append(node)
        self._nodes.sort(key=lambda x: x.identifier)

    def remove_node(self, identifier: int) -> None:
        """
        Remove a node and rebalance the tree.
        """
        for i, killed_node in enumerate(self._nodes):
            if killed_node.identifier == identifier:
                break
        assert i < len(self._nodes) - 1  # cannot delete last node
        
        # rebalancing
        inheriting_node = self._nodes[i + 1]
        for key, value in killed_node.bulk_pairs(inheriting_node.identifier):
            inheriting_node.set(key, value)
        
        # killing node
        self._nodes.remove(killed_node)
        del killed_node

    def set(self, key: str, value: str) -> None:
        key = self._hash_function(key)
        for node in self._nodes:  # as nodes are sorted the first hit is our desired match
            if node.identifier >= key:
                return node.set(key, value)
        assert False
    
    def get(self, key: str) -> str:
        """
        Perform a search for the node that may contain the value we need.
        """
        key = self._hash_function(key)
        for node in self._nodes:
            if node.identifier >= key:
                return node.get(key)
        assert False
    
    def get_identifier(self, key: str) -> int:
        """
        Return the identifier of the node that the key resides on
        """
        key = self._hash_function(key)
        for node in self._nodes:
            if node.identifier >= key:
                return node.identifier
        assert False

In [130]:
ring = HashRing(10 ** 8)

ring.add_node(10 ** 8)
ring.set('a', 'a')
ring.set('b', 'b')
ring.set('cneiuouwenfouwnefonowuenfounwef', 'cneiuouwenfouwnefonowuenfounwef')
ring.set('d', 'd')
ring.set('aasdiqwdbqowhdoqwd', 'aasdiqwdbqowhdoqwd')
ring.set('127hih12brilbo7912r', '127hih12brilbo7912r')
ring.set('iev3h98223nof23', 'iev3h98223nof23')
ring.set('r56', 'r56')

In [131]:
ring._nodes[0]._data

{58726497: 'a',
 95795343: 'b',
 25912199: 'cneiuouwenfouwnefonowuenfounwef',
 61147053: 'd',
 1779395: 'aasdiqwdbqowhdoqwd',
 81549590: '127hih12brilbo7912r',
 48536630: 'iev3h98223nof23',
 69617187: 'r56'}

In [133]:
ring.add_node(50_000_000)

In [135]:
ring._nodes[0]._data

{25912199: 'cneiuouwenfouwnefonowuenfounwef',
 1779395: 'aasdiqwdbqowhdoqwd',
 48536630: 'iev3h98223nof23'}

In [134]:
ring.get('iev3h98223nof23')
ring.get_identifier('iev3h98223nof23')

50000000

In [136]:
ring.remove_node(50_000_000)

In [137]:
ring._nodes[0]._data

{58726497: 'a',
 95795343: 'b',
 61147053: 'd',
 81549590: '127hih12brilbo7912r',
 69617187: 'r56',
 25912199: 'cneiuouwenfouwnefonowuenfounwef',
 1779395: 'aasdiqwdbqowhdoqwd',
 48536630: 'iev3h98223nof23'}